Chapter 6 code and figures

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from matplotlib.pyplot import subplots
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

In [2]:
Credit = pd.read_csv('data/Credit.csv')
Credit['Own']=(Credit['Own']=='Yes').astype(int)
Credit['Student']=(Credit['Student']=='Yes').astype(int)
Credit['Married']=(Credit['Married']=='Yes').astype(int)
Credit['South']=(Credit['Region']=='South').astype(int)
Credit['West']=(Credit['Region']=='West').astype(int)
Credit

,Income,Limit,Rating,Cards,Age,Education,Own,Student,Married,Region,Balance,South,West
0,14.891,3606,283,2,34,11,0,0,1,South,333,1,0
1,106.025,6645,483,3,82,15,1,1,1,West,903,0,1
2,104.593,7075,514,4,71,11,0,0,0,West,580,0,1
3,148.924,9504,681,3,36,11,1,0,0,West,964,0,1
4,55.882,4897,357,2,68,16,0,0,1,South,331,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.096,4100,307,3,32,13,0,0,1,South,560,1,0
396,13.364,3838,296,5,65,17,0,0,0,East,480,0,0
397,57.872,4171,321,5,67,12,1,0,1,South,138,1,0
398,37.728,2525,192,1,44,13,0,0,1,South,0,1,0


First fit the model with all of the predictor variables

In [3]:
y = Credit['Balance']
terms = Credit.columns.drop(['Region','Balance'])
X = MS(terms).fit_transform(Credit)
model = sm.OLS(y, X)
results = model.fit()
# Different ways of summary reseults. The second contains a lot of information.
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                Balance   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     750.3
Date:                Mon, 04 Mar 2024   Prob (F-statistic):          1.11e-253
Time:                        14:19:43   Log-Likelihood:                -2398.7
No. Observations:                 400   AIC:                             4821.
Df Residuals:                     388   BIC:                             4869.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   -479.2079     35.774    -13.395      0.0

## Alg 6.2: Forward Stepwise Selection to k = 4 ##

First, see which predictor has the most predictive power

In [4]:
R_squared = []
terms_fixed = []
TSS = np.linalg.norm(y-y.mean())**2
for col in terms:
    X = MS([col]).fit_transform(Credit)
    model = sm.OLS(y, X)
    results = model.fit()
    R_squared.append(1-np.linalg.norm(results.resid)**2/TSS)
    print('term: {}, R_squared: {}'.format(col, R_squared[-1]))

terms_fixed.append(terms[R_squared.index(max(R_squared))])
print(terms_fixed)

term: Income, R_squared: 0.21497731013240484
term: Limit, R_squared: 0.7425221799818016
term: Rating, R_squared: 0.7458484180585037
term: Cards, R_squared: 0.007474700008969104
term: Age, R_squared: 3.3676612274380346e-06
term: Education, R_squared: 6.498901491658327e-05
term: Own, R_squared: 0.00046113296449656893
term: Student, R_squared: 0.06709008988708953
term: Married, R_squared: 3.218849124542178e-05
term: South, R_squared: 1.0813056131042664e-05
term: West, R_squared: 9.62799739034903e-05
['Rating']


Now, add a second predictor to the one computed above

In [5]:
terms.drop(terms_fixed[0]) # remove the first term from the list (the one with the highest R_squared)
R_squared = []
for col in terms: # iterate over the remaining terms
    X = MS([terms_fixed[0],col]).fit_transform(Credit) # add the first term to the current term
    model = sm.OLS(y, X)
    results = model.fit()
    R_squared.append(1-np.linalg.norm(results.resid)**2/TSS)
    print('term: {}, R_squared: {}'.format(col, R_squared[-1]))

terms_fixed.append(terms[R_squared.index(max(R_squared))])
print(terms_fixed)

term: Income, R_squared: 0.8751179476994355
term: Limit, R_squared: 0.745942796101409
term: Rating, R_squared: 0.7458484180585037
term: Cards, R_squared: 0.7474915260521078
term: Age, R_squared: 0.7535447719888835
term: Education, R_squared: 0.7461714278077378
term: Own, R_squared: 0.7460389021078521
term: Student, R_squared: 0.8138489985847958
term: Married, R_squared: 0.7472499974386341
term: South, R_squared: 0.7458540089138774
term: West, R_squared: 0.7463017482302219
['Rating', 'Income']


Add a third predictor 

In [18]:
terms.drop(terms_fixed[1])
R_squared = []
for col in terms:
    X = MS([terms_fixed[0],terms_fixed[1],col]).fit_transform(Credit)
    model = sm.OLS(y, X)
    results = model.fit()
    R_squared.append(1-np.linalg.norm(results.resid)**2/TSS)

terms_fixed.append(terms[R_squared.index(max(R_squared))])
print(terms_fixed)

['Rating', 'Income', 'Student']


And finally, add a fourth predictor. 

In [19]:
terms = terms.drop(terms[2])
R_squared = []
for col in terms:
    X = MS([terms_fixed[0],terms_fixed[1],terms_fixed[2],col]).fit_transform(Credit)
    model = sm.OLS(y, X)
    results = model.fit()
    R_squared.append(1-np.linalg.norm(results.resid)**2/TSS)

terms_fixed.append(terms[R_squared.index(max(R_squared))])
print(terms_fixed)

['Rating', 'Income', 'Student', 'Limit']


Note that this agrees with Table 6.1